<a href="https://colab.research.google.com/github/nileshkumar2368/AI_ChatBot/blob/main/AI_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask-ngrok transformers sentence-transformers nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import json
import nltk
import torch
import numpy as np
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
faq_data = """
{
  "faqs": [
    {"id": 1, "question": "How do I reset my password?", "answer": "Go to the login page, click on 'Forgot Password', and follow the instructions."},
    {"id": 2, "question": "What is the refund policy?", "answer": "You can request a refund within 30 days of purchase with a valid receipt."},
    {"id": 3, "question": "How can I contact customer support?", "answer": "You can reach us via email at support@example.com or call 1-800-555-5555."},
    {"id": 4, "question": "Do you offer international shipping?", "answer": "Yes, we ship to over 50 countries. Shipping costs vary based on location."},
    {"id": 5, "question": "How can I track my order?", "answer": "Log in to your account and go to 'My Orders' to track your shipment."},

    {"id": 6, "question": "What payment methods do you accept?", "answer": "We accept Visa, MasterCard, PayPal, and Apple Pay."},
    {"id": 7, "question": "How do I apply for a job at your company?", "answer": "Visit our careers page and submit an online application."},
    {"id": 8, "question": "Can I change my order after placing it?", "answer": "Orders can be modified within 24 hours. Contact support for assistance."},
    {"id": 9, "question": "Do you provide discounts for bulk orders?", "answer": "Yes, we offer discounts on bulk purchases. Contact our sales team for details."},
    {"id": 10, "question": "How do I unsubscribe from marketing emails?", "answer": "Click the 'Unsubscribe' link at the bottom of any marketing email."},

    {"id": 11, "question": "Can I return an item after 60 days?", "answer": "No, our return policy only allows for returns within 30 days."},
    {"id": 12, "question": "What should I do if my order is damaged?", "answer": "Contact customer support with photos of the damage within 48 hours of delivery."},
    {"id": 13, "question": "How long does shipping take?", "answer": "Shipping time varies based on your location and shipping method."},
    {"id": 14, "question": "Do you have a loyalty program?", "answer": "Yes, we offer reward points for purchases, which can be redeemed for discounts."},
    {"id": 15, "question": "What are your business hours?", "answer": "Our customer support team is available Monday to Friday, 9 AM - 6 PM EST."},

    {"id": 16, "question": "Can I cancel my subscription anytime?", "answer": "Yes, you can cancel your subscription anytime from your account settings."},
    {"id": 17, "question": "Do you offer student discounts?", "answer": "Yes, we offer a 10% discount for students. Verify your student status to apply."},
    {"id": 18, "question": "Is my personal data secure?", "answer": "Yes, we follow industry best practices to protect your personal data and privacy."},
    {"id": 19, "question": "How do I report a bug or issue?", "answer": "Please submit a bug report via our support portal or email us at bugs@example.com."},
    {"id": 20, "question": "How can I upgrade my account?", "answer": "Go to account settings and select 'Upgrade Plan' to choose a higher-tier plan."},

    {"id": 21, "question": "Can I use your service on multiple devices?", "answer": "Yes, your account can be accessed from multiple devices simultaneously."},
    {"id": 22, "question": "Do you support two-factor authentication?", "answer": "Yes, you can enable 2FA in your security settings for added account protection."},
    {"id": 23, "question": "How do I delete my account permanently?", "answer": "To delete your account, go to account settings and select 'Delete Account'. This action cannot be undone."},
    {"id": 24, "question": "Are your products covered by warranty?", "answer": "Yes, all our products come with a one-year warranty against manufacturing defects."},
    {"id": 25, "question": "Can I request a product demo?", "answer": "Yes, schedule a free product demo by filling out the form on our website."},

    {"id": 100, "question": "Do you support cryptocurrency payments?", "answer": "Currently, we do not support cryptocurrency payments."},
    {"id": 101, "question": "How do I join your affiliate program?", "answer": "Sign up for our affiliate program on our website to start earning commissions."},
    {"id": 102, "question": "Can I customize the software to my needs?", "answer": "We offer customization options for enterprise clients. Contact sales for details."},
    {"id": 103, "question": "Do you provide training for new users?", "answer": "Yes, we provide onboarding training and user guides for new customers."},
    {"id": 104, "question": "Can I integrate your service with third-party apps?", "answer": "Yes, we offer API integrations with several third-party applications."},

    {"id": 200, "question": "Is your customer support available 24/7?", "answer": "We offer 24/7 email support. Live chat and phone support are available during business hours."},
    {"id": 201, "question": "Do you provide enterprise solutions?", "answer": "Yes, we offer customized enterprise solutions based on business needs."},
    {"id": 202, "question": "What happens if my payment fails?", "answer": "If your payment fails, you will receive a notification. Please update your payment method."},
    {"id": 203, "question": "Can I reschedule a delivery?", "answer": "Yes, you can request a delivery reschedule by contacting our logistics team."},
    {"id": 204, "question": "Do you offer refunds for digital products?", "answer": "Refunds for digital products are subject to approval based on our terms of service."},

    {"id": 300, "question": "Can I request additional storage on my account?", "answer": "Yes, you can purchase additional storage through your account settings."},
    {"id": 301, "question": "How do I report inappropriate content on your platform?", "answer": "Use the 'Report' button next to the content or contact our moderation team."},
    {"id": 302, "question": "Do you offer multi-user accounts for teams?", "answer": "Yes, we provide team accounts with role-based access control."},
    {"id": 303, "question": "Can I pause my subscription instead of canceling?", "answer": "Yes, you can pause your subscription for up to 3 months."},
    {"id": 304, "question": "How do I get notified about new features?", "answer": "Enable product updates in your notification settings to receive updates."}
  ]
}



"""

faq_dict = json.loads(faq_data)
faqs = faq_dict['faqs']


In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Precompute FAQ question embeddings
faq_questions = [faq["question"] for faq in faqs]
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def find_best_match(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]
    best_score_idx = torch.argmax(scores).item()
    confidence = scores[best_score_idx].item()

    if confidence < 0.50:
        return "I'm not sure about this, let me find more details.", confidence

    return faqs[best_score_idx]["answer"], confidence


In [7]:
biased_phrases = ["best", "top-rated", "number one", "highly recommended", "industry-leading"]

def detect_bias(answer):
    words = answer.split()
    neutral_answer = []
    for word in words:
        if word.lower() in biased_phrases:
            neutral_answer.append("[neutral]")
        else:
            neutral_answer.append(word)
    return " ".join(neutral_answer)



In [8]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/chat", methods=["POST"])
def chatbot():
    data = request.json
    user_query = data.get("query", "")

    if not user_query:
        return jsonify({"error": "Query missing"}), 400

    answer, confidence = find_best_match(user_query)
    unbiased_answer = detect_bias(answer)

    return jsonify({"answer": unbiased_answer, "confidence": confidence})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [ ]:
import requests

url = "http://127.0.0.1:5000/chat"
data = {"query": "What is the refund policy?"}

response = requests.post(url, json=data)
print(response.json())


In [9]:
!pip install flask flask-ngrok transformers sentence-transformers nltk


In [10]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import json
import torch
from sentence_transformers import SentenceTransformer, util

# Initialize Flask App
app = Flask(__name__)
run_with_ngrok(app)  # Enables Ngrok

# Load FAQ Data
faq_data = """
{
  "faqs": [
    {"id": 1, "question": "How do I reset my password?", "answer": "Go to the login page, click on 'Forgot Password', and follow the instructions."},
    {"id": 2, "question": "What is the refund policy?", "answer": "You can request a refund within 30 days of purchase with a valid receipt."}
  ]
}
"""
faq_dict = json.loads(faq_data)
faqs = faq_dict['faqs']

# Load NLP Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
faq_questions = [faq["question"] for faq in faqs]
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

# Function to Find Best Match
def find_best_match(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]
    best_score_idx = torch.argmax(scores).item()
    confidence = scores[best_score_idx].item()

    if confidence < 0.50:
        return "I'm not sure about this, let me find more details.", confidence

    return faqs[best_score_idx]["answer"], confidence

# Flask API Route
@app.route("/chat", methods=["POST"])
def chatbot():
    data = request.json
    user_query = data.get("query", "")

    if not user_query:
        return jsonify({"error": "Query missing"}), 400

    answer, confidence = find_best_match(user_query)
    return jsonify({"answer": answer, "confidence": confidence})

# Start Flask App
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [11]:
!pip install flask pyngrok



In [12]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import json
import torch
from sentence_transformers import SentenceTransformer, util

# Initialize Flask App
app = Flask(__name__)

# Start ngrok manually
public_url = ngrok.connect(5000).public_url
print(f"Ngrok Tunnel: {public_url}")

# Load FAQ Data
faq_data = """
{
  "faqs": [
    {"id": 1, "question": "How do I reset my password?", "answer": "Go to the login page, click on 'Forgot Password', and follow the instructions."},
    {"id": 2, "question": "What is the refund policy?", "answer": "You can request a refund within 30 days of purchase with a valid receipt."}
  ]
}
"""
faq_dict = json.loads(faq_data)
faqs = faq_dict['faqs']

# Load NLP Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
faq_questions = [faq["question"] for faq in faqs]
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

# Function to Find Best Match
def find_best_match(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]
    best_score_idx = torch.argmax(scores).item()
    confidence = scores[best_score_idx].item()

    if confidence < 0.50:
        return "I'm not sure about this, let me find more details.", confidence

    return faqs[best_score_idx]["answer"], confidence

# Flask API Route
@app.route("/chat", methods=["POST"])
def chatbot():
    data = request.json
    user_query = data.get("query", "")

    if not user_query:
        return jsonify({"error": "Query missing"}), 400

    answer, confidence = find_best_match(user_query)
    return jsonify({"answer": answer, "confidence": confidence})

# Start Flask App
if __name__ == "__main__":
    app.run(port=5000)


ERROR:pyngrok.process.ngrok:t=2025-02-23T10:59:58+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-23T10:59:58+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-23T10:59:58+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [13]:
!pip install flask gradio sentence-transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import gradio as gr
import json
import torch
from sentence_transformers import SentenceTransformer, util

# Load FAQ Data
faq_data = """
{
  "faqs": [
    {"id": 1, "question": "How do I reset my password?", "answer": "Go to the login page, click on 'Forgot Password', and follow the instructions."},
    {"id": 2, "question": "What is the refund policy?", "answer": "You can request a refund within 30 days of purchase with a valid receipt."},
    {"id": 3, "question": "How can I contact customer support?", "answer": "You can reach us via email at support@example.com or call 1-800-555-5555."},
    {"id": 4, "question": "Do you offer international shipping?", "answer": "Yes, we ship to over 50 countries. Shipping costs vary based on location."},
    {"id": 5, "question": "How can I track my order?", "answer": "Log in to your account and go to 'My Orders' to track your shipment."},

    {"id": 6, "question": "What payment methods do you accept?", "answer": "We accept Visa, MasterCard, PayPal, and Apple Pay."},
    {"id": 7, "question": "How do I apply for a job at your company?", "answer": "Visit our careers page and submit an online application."},
    {"id": 8, "question": "Can I change my order after placing it?", "answer": "Orders can be modified within 24 hours. Contact support for assistance."},
    {"id": 9, "question": "Do you provide discounts for bulk orders?", "answer": "Yes, we offer discounts on bulk purchases. Contact our sales team for details."},
    {"id": 10, "question": "How do I unsubscribe from marketing emails?", "answer": "Click the 'Unsubscribe' link at the bottom of any marketing email."},

    {"id": 11, "question": "Can I return an item after 60 days?", "answer": "No, our return policy only allows for returns within 30 days."},
    {"id": 12, "question": "What should I do if my order is damaged?", "answer": "Contact customer support with photos of the damage within 48 hours of delivery."},
    {"id": 13, "question": "How long does shipping take?", "answer": "Shipping time varies based on your location and shipping method."},
    {"id": 14, "question": "Do you have a loyalty program?", "answer": "Yes, we offer reward points for purchases, which can be redeemed for discounts."},
    {"id": 15, "question": "What are your business hours?", "answer": "Our customer support team is available Monday to Friday, 9 AM - 6 PM EST."},

    {"id": 16, "question": "Can I cancel my subscription anytime?", "answer": "Yes, you can cancel your subscription anytime from your account settings."},
    {"id": 17, "question": "Do you offer student discounts?", "answer": "Yes, we offer a 10% discount for students. Verify your student status to apply."},
    {"id": 18, "question": "Is my personal data secure?", "answer": "Yes, we follow industry best practices to protect your personal data and privacy."},
    {"id": 19, "question": "How do I report a bug or issue?", "answer": "Please submit a bug report via our support portal or email us at bugs@example.com."},
    {"id": 20, "question": "How can I upgrade my account?", "answer": "Go to account settings and select 'Upgrade Plan' to choose a higher-tier plan."},

    {"id": 21, "question": "Can I use your service on multiple devices?", "answer": "Yes, your account can be accessed from multiple devices simultaneously."},
    {"id": 22, "question": "Do you support two-factor authentication?", "answer": "Yes, you can enable 2FA in your security settings for added account protection."},
    {"id": 23, "question": "How do I delete my account permanently?", "answer": "To delete your account, go to account settings and select 'Delete Account'. This action cannot be undone."},
    {"id": 24, "question": "Are your products covered by warranty?", "answer": "Yes, all our products come with a one-year warranty against manufacturing defects."},
    {"id": 25, "question": "Can I request a product demo?", "answer": "Yes, schedule a free product demo by filling out the form on our website."},

    {"id": 100, "question": "Do you support cryptocurrency payments?", "answer": "Currently, we do not support cryptocurrency payments."},
    {"id": 101, "question": "How do I join your affiliate program?", "answer": "Sign up for our affiliate program on our website to start earning commissions."},
    {"id": 102, "question": "Can I customize the software to my needs?", "answer": "We offer customization options for enterprise clients. Contact sales for details."},
    {"id": 103, "question": "Do you provide training for new users?", "answer": "Yes, we provide onboarding training and user guides for new customers."},
    {"id": 104, "question": "Can I integrate your service with third-party apps?", "answer": "Yes, we offer API integrations with several third-party applications."},

    {"id": 200, "question": "Is your customer support available 24/7?", "answer": "We offer 24/7 email support. Live chat and phone support are available during business hours."},
    {"id": 201, "question": "Do you provide enterprise solutions?", "answer": "Yes, we offer customized enterprise solutions based on business needs."},
    {"id": 202, "question": "What happens if my payment fails?", "answer": "If your payment fails, you will receive a notification. Please update your payment method."},
    {"id": 203, "question": "Can I reschedule a delivery?", "answer": "Yes, you can request a delivery reschedule by contacting our logistics team."},
    {"id": 204, "question": "Do you offer refunds for digital products?", "answer": "Refunds for digital products are subject to approval based on our terms of service."},

    {"id": 300, "question": "Can I request additional storage on my account?", "answer": "Yes, you can purchase additional storage through your account settings."},
    {"id": 301, "question": "How do I report inappropriate content on your platform?", "answer": "Use the 'Report' button next to the content or contact our moderation team."},
    {"id": 302, "question": "Do you offer multi-user accounts for teams?", "answer": "Yes, we provide team accounts with role-based access control."},
    {"id": 303, "question": "Can I pause my subscription instead of canceling?", "answer": "Yes, you can pause your subscription for up to 3 months."},
    {"id": 304, "question": "How do I get notified about new features?", "answer": "Enable product updates in your notification settings to receive updates."}
  ]
}



"""
faq_dict = json.loads(faq_data)
faqs = faq_dict['faqs']

# Load NLP Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
faq_questions = [faq["question"] for faq in faqs]
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

# Function to Find Best Match
def find_best_match(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]
    best_score_idx = torch.argmax(scores).item()
    confidence = scores[best_score_idx].item()

    if confidence < 0.50:
        return "I'm not sure about this, let me find more details."

    return faqs[best_score_idx]["answer"]

# Gradio Interface
def chatbot_interface(user_input):
    response = find_best_match(user_input)
    return response

# Launch Gradio App
iface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="FAQ Chatbot")
iface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e119c5df386f1badfa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
